In [ ]:
%pip install openai pinecone-client datasets faiss flask


In [ ]:
import os
from flask import Flask, request, jsonify
from retriever import retrieve_context
from generator import generate_response

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get("message", "")
    retrieved_context = retrieve_context(user_input)
    response = generate_response(user_input, retrieved_context)
    return jsonify({"response": response})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)


In [ ]:
import pinecone
import os
import faiss
import numpy as np
from datasets import load_dataset
from openai import OpenAI

PINECONE_API_KEY = "pnc-1234abcd5678xyz"
PINECONE_ENV = "us-west1-gcp"

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
index_name = "chat-memory"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=768)

index = pinecone.Index(index_name)

client = OpenAI(api_key="sk-1234abcd5678xyz")

def embed_text(text):
    response = client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response["data"][0]["embedding"]

def store_data():
    dataset = load_dataset("ag_news", split="train[:100]")
    for data in dataset:
        text = data["text"]
        vector = embed_text(text)
        index.upsert([(text, vector)])

store_data()


In [ ]:
import pinecone
import numpy as np
from vector_store import index, embed_text

def retrieve_context(query):
    query_vector = embed_text(query)
    results = index.query(vector=query_vector, top_k=5, include_metadata=True)
    return " ".join([match["id"] for match in results["matches"]]) if results.get("matches") else ""


In [ ]:
from openai import OpenAI

client = OpenAI(api_key="sk-1234abcd5678xyz")

def generate_response(user_input, context):
    prompt = f"User: {user_input}\nContext: {context}\nAssistant:"
    response = client.chat.completions.create(model="gpt-4", messages=[{"role": "system", "content": prompt}])
    return response.choices[0].message["content"]


In [ ]:
python app.py


In [ ]:
POST http://localhost:5000/chat
{
    "message": "Tell me about AI ethics."
}
